In [4]:
import torch 
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

from pytorch.dataloader import get_dataloaders
from pytorch.lstm import LSTM

In [3]:
device = torch.device("cpu")
base_df_dir = "./data/"

In [4]:
BS = 32

train_iter, valid_iter, test_iter = get_dataloaders(base_df_dir, BS, device)

100%|██████████| 4029/4029 [00:01<00:00, 3014.77it/s]


In [17]:
def train(
    model,
    optimizer,
    criterion=nn.BCELoss(),
    train_loader=train_iter,
    valid_loader=valid_iter,
    num_epochs=5,
    eval_every=5,
):
  run_loss = 0.0
  valid_run_loss = 0.0
  global_step = 0
  train_loss_list = []
  valid_loss_list = []
  global_step_list = []
  ave_train_loss = None
  ave_valid_loss = None

  model.train()
  for epoch in range(num_epochs):
    with tqdm(train_loader, unit="batch") as tbatch:
      for labels, titletext, titletext_len in tbatch:
        tbatch.set_description(f"Epoch {epoch}")

        output = model(titletext, titletext_len)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        run_loss += loss.item()
        global_step += 1

        tbatch.set_postfix(step=f"{ global_step }/{(num_epochs * len(train_loader)) }",
                           train_loss=ave_train_loss,
                           valid_loss=ave_valid_loss)

        if global_step % eval_every == 0:
          model.eval()
          with torch.no_grad():
            for labels, titletext, titletext_len in valid_loader:
              labels = labels.to(device)
              titletext = titletext.to(device)
              titletext_len = titletext_len.to(device)

              output = model(titletext, titletext_len)
              loss = criterion(output, labels)

              valid_run_loss += loss.item()

          ave_train_loss = run_loss / eval_every
          ave_valid_loss = run_loss / len(valid_loader)
          train_loss_list.append(ave_train_loss)
          valid_loss_list.append(ave_valid_loss)
          global_step_list.append(global_step)

          run_loss, valid_run_loss = 0.0, 0.0

          model.train()
          tbatch.set_postfix(step=f"{ global_step }/{(num_epochs * len(train_loader)) }",
                             train_loss=loss.item(),
                             valid_loss=ave_valid_loss)

  print("Finished training")


model = LSTM(vocab=vocab).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train(model=model, optimizer=optimizer, num_epochs=10)


Epoch 0:   5%|▍         | 6/126 [01:24<28:11, 14.10s/batch, step=6/1260, train_loss=0.681, valid_loss=0.106]

In [ ]:
state_dict = {
  "model_state_dict": model.state_dict(),
  "optimizer_state_dict": optimizer.state_dict(),
}

torch.save(state_dict, "../common-data/fake-news/")